In [ ]:
import json
import numpy as np
from itertools import product

# Template Notebook

In [ ]:
ALPHA = 'abcdefghijklmnopqrstuvwxyz'
B = "⚫"
W = "⚪"
PT = "╶╴"
SP = "╺╸"

BLACK = '\033[30m'      # ]
GRAY = '\033[90m'       # ]
YELLOW = '\033[33m'     # ]
BG_YELLOW = '\033[43m'  # ]
RESET = '\033[0m'       # ]


class Board:
    def __init__(self, size: int):
        self.size = size
        self.state = np.zeros((size,size), dtype=int)

    def play(self, player: int, x: int, y: int) -> None:
        self.state[y, x] = player

    def plaintext_board(self) -> str:
        star_points = np.zeros((self.size,self.size), dtype=int)
        corners = [j for i in range(3) if (j:=((s:=2+(self.size>9))+(2*s*i))) < self.size]
        pts = [(f:=self.size//2, f)] + list(product(corners, repeat=2))
        star_points[*zip(*pts)] = -1

        board = self.state.copy()
        mask = ~self.state.astype(bool)
        board[mask] = star_points[mask]

        joined = ' '.join(list(ALPHA.replace('i', '')[:size])).upper()
        rows = [col_label:=f"{YELLOW}{(d:='-' * s)}{BLACK}{joined} {YELLOW}{d}"]

        for r, input_row in enumerate(board, 1):
            row = ''.join([(PT,B,W,SP)[i] for i in input_row])
            num = str(size - r + 1)
            lnum = num.rjust(int(len(str(size))))
            rnum = num.ljust(int(len(str(size))))
            rows.append(f'{BLACK}{lnum} {GRAY}{row} {BLACK}{rnum}')

        rows.append(col_label)
        rows = [f'{BG_YELLOW}{row}{RESET}' for row in rows]

        return '\n'.join(rows)

    def __str__(self):
        return self.plaintext_board()

for size in [13]:
    b = Board(size)
    b.play(1, 4, 4)
    b.play(2, 3, 2)
    print(b)

In [ ]:
import os
import requests

BASE_URL = "https://online-go.com/api/v1/games"
game_id = "68984609"
request_url = os.path.join(BASE_URL, game_id, 'sgf')

r = requests.get(request_url)

print(r)
print(r.url)

# print(json.dumps(r.json()))

In [ ]:
import re

sgf = r.content.decode('utf-8')
text = sgf.strip(')').split(';')
meta = dict(re.findall(r'(\w+)\[(.*?)\]\n?', text[1]))
moves = [tuple(m.strip('\n()]').split('[')) for m in text[2:]]

print(meta)
print(moves)

In [ ]:
b = Board(13)
for player, move in moves:
    player_num = 'BW'.index(player) + 1

    if move:  # an empty move is a pass
        x, y = (ALPHA.index(c) for c in move)
        b.play(player_num, x, y)

print(b)